In [6]:
import elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain.schema import Document
# from sentence_transformers import SentenceTransformer

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# Tạo mô hình embedding
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Kết nối đến Elasticsearch
es_client = elasticsearch.Elasticsearch(
    hosts=["http://localhost:9200"],
    http_auth=("phuongln10", "gptllm"),  # Sử dụng http_auth để cung cấp thông tin đăng nhập
    max_retries=10
)

# Kết nối đến Elasticsearch
# es_store = ElasticsearchStore(
#     es_api_key= "WGk4NklKRUJMVHJPcjBORWUzNU86aDNNVUJxUFBRa0tDVkNrWlgyYWZiZw==",
#     es_cloud_id="0d88bad6b77d4efe935c19ac5b44b113:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ1YzgwMzljY2Q4NDE0Y2I0ODJiOTcyZDg4ODgyZjkzMyRhMTAyMGM0MGY4OTg0YTBiYmNjZjczZmQ0NmI5MmM2NA==",
#     es_connection=es_client
# )

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader(r"C:\Users\Admin\Documents\F_GPT_Demo\build_database\state_of_the_union.txt", encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

db = ElasticsearchStore.from_documents(
    docs,
    embedding_function,
    index_name="test-basic",
    es_api_key= "WGk4NklKRUJMVHJPcjBORWUzNU86aDNNVUJxUFBRa0tDVkNrWlgyYWZiZw==",
    es_cloud_id="0d88bad6b77d4efe935c19ac5b44b113:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ1YzgwMzljY2Q4NDE0Y2I0ODJiOTcyZDg4ODgyZjkzMyRhMTAyMGM0MGY4OTg0YTBiYmNjZjczZmQ0NmI5MmM2NA==",
    es_connection=es_client
)

db.client.indices.refresh(index="test-basic")

query = "What did the president say about Ketanji Brown Jackson"
results = db.similarity_search(query)
print(results)

c:\Users\Admin\Documents\F_GPT_Demo\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\Admin\AppData\Local\Temp\ipykernel_23304\558707282.py:14: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = elasticsearch.Elasticsearch(


[Document(metadata={'source': 'C:\\Users\\Admin\\Documents\\F_GPT_Demo\\build_database\\state_of_the_union.txt'}, page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'), Document(metadata={'source': 'C:\\Users\\Admin\\Documents\\F_GPT_Demo\\build_database\\state_of_the_union.txt'}, page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming mi

In [7]:
# Adding metadata to documents
for i, doc in enumerate(docs):
    doc.metadata["date"] = f"{range(2010, 2020)[i % 10]}-01-01"
    doc.metadata["rating"] = range(1, 6)[i % 5]
    doc.metadata["author"] = ["John Doe", "Jane Doe"][i % 2]

db = ElasticsearchStore.from_documents(
    docs,
    embedding_function,
    es_api_key= "WGk4NklKRUJMVHJPcjBORWUzNU86aDNNVUJxUFBRa0tDVkNrWlgyYWZiZw==",
    es_cloud_id="0d88bad6b77d4efe935c19ac5b44b113:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ1YzgwMzljY2Q4NDE0Y2I0ODJiOTcyZDg4ODgyZjkzMyRhMTAyMGM0MGY4OTg0YTBiYmNjZjczZmQ0NmI5MmM2NA==",
    es_connection=es_client, index_name="test-metadata"
)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[2].metadata)

{'source': 'C:\\Users\\Admin\\Documents\\F_GPT_Demo\\build_database\\state_of_the_union.txt', 'date': '2016-01-01', 'rating': 2, 'author': 'John Doe'}


In [13]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[2].metadata)

{'source': 'C:\\Users\\Admin\\Documents\\F_GPT_Demo\\build_database\\state_of_the_union.txt', 'date': '2015-01-01', 'rating': 1, 'author': 'Jane Doe'}


In [12]:
loader = TextLoader(r"C:\Users\Admin\Documents\F_GPT_Demo\build_database\state_of_the_union.txt", encoding='utf-8')
documents = loader.load()
print(type(documents))

<class 'list'>
